In [1]:
from vizdoom import DoomGame  
import random
import time
import numpy as np
import gym
from gym import Env
from gym.spaces import Discrete, Box
import cv2
from matplotlib import pyplot as plt
import os
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3 import PPO

In [2]:
###################
##   CONSTANTS   ##
###################
ACTION_NUM = 20
EPISODES_NUM = 10
AGENT_MODEL_PATH_PREFIX = './agents/agent_for_'
TENSORBOARD_LOG_PATH_PREFIX = './logs/logs_for_'
CURRENT_CONFIGURATION_INDEX = 3

configurations = [{
                    'name': 'basic',
                    'scenarioConfigFilePath': 'VizDoom/scenarios/basic.cfg',
                    'actionNumber': 3,
                  }, {
                    'name': 'defend_the_center',
                    'scenarioConfigFilePath': 'VizDoom/scenarios/defend_the_center.cfg',
                    'actionNumber': 3,
                  }, {
                    'name': 'deadly_corridor',
                    'scenarioConfigFilePath': 'VizDoom/scenarios/deadly_corridor_label_buffer.cfg',
                    'actionNumber': 7,
                  }, {
                    'name': 'deathmatch',
                    'scenarioConfigFilePath': 'VizDoom/scenarios/deathmatch.cfg',
                    'actionNumber': 20,
                  }]


In [3]:
class VizDoomGym(Env):
  def __init__(self, envConfig, render=False):
    super().__init__()
    self.game = DoomGame()
    self.game.load_config(envConfig["scenarioConfigFilePath"])
    self.game.set_window_visible(render)
    self.game.init()

    self.action_number = envConfig["actionNumber"]
    self.action_space = Discrete(self.action_number)
    self.observation_space = Box(0, 255, [100, 160, 1], np.uint8)

  def close(self):
    self.game.close()
  
  def step(self, action):
    actions = np.identity(self.action_number, dtype=np.uint8)
    actionReward = self.game.make_action(actions[action], 5)

    done = self.game.is_episode_finished()
    state = self.game.get_state()
  
    if not state:
        return np.zeros(self.observation_space.shape), actionReward, done, {"damage_taken": 0, "hitcount": 0, "ammo": 0}
      
  


    
    
    img = self.grayscale(state.screen_buffer)
    # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    return img, actionReward, done, {}
  
  def reset(self):
    self.game.new_episode()
    state = self.game.get_state()
    self.rewardsObject = {
      'damage_taken': 0,
      'hitcount': 0,
      'ammo': 52,
      'armor': 0,
      'killcount': 0
    }
    return self.grayscale(state.screen_buffer)
    
  
  def render():
    pass
  
  def grayscale(self, observation):
    grayscaled = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(grayscaled, (160, 100), cv2.INTER_CUBIC)
    return np.reshape(resized, (100, 160, 1))
  

In [4]:
env = VizDoomGym(configurations[CURRENT_CONFIGURATION_INDEX], render=True)

In [5]:
from stable_baselines3.common.evaluation import evaluate_policy
model = PPO.load(f"{AGENT_MODEL_PATH_PREFIX}{configurations[CURRENT_CONFIGURATION_INDEX]['name']}/model_1170000")

In [6]:
from IPython.display import clear_output

for episode in range(10):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        # clear_output()
        # plt.imshow(obs)
        # plt.show()
        time.sleep(0.05555)
    time.sleep(1)

ViZDoomUnexpectedExitException: Controlled ViZDoom instance exited unexpectedly.

In [32]:
evaluate_policy(model, env, n_eval_episodes=10)

e:\Programs\Anaconda\envs\py37\lib\site-packages\stable_baselines3\common\evaluation.py:71: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


ViZDoomIsNotRunningException: Controlled ViZDoom instance is not running or not ready.

In [27]:
env.close()

In [7]:
from stable_baselines3.common.evaluation import evaluate_policy
model = PPO.load(f"{AGENT_MODEL_PATH_PREFIX}{configurations[CURRENT_CONFIGURATION_INDEX]['name']}/model_1700000")
from IPython.display import clear_output

for episode in range(10):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        clear_output()
        plt.imshow(obs)
        plt.show()
        time.sleep(0.05)
    time.sleep(1)

ValueError: too many values to unpack (expected 4)